In [1]:
from nltk.tokenize import RegexpTokenizer
import cPickle as pickle
import string
import os
import re
import urllib2

import numpy as np
import random
import time

import json

import logging
from logging import info

from multiprocessing import Pool as ThreadPool
import itertools

import xml.etree.ElementTree as ET

import nltk

from thesis.utils.text import get_sentences, sentence_wordtokenizer

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
# root_location = "/mnt/data2/shalaby/"
root_location = "/home/local/shalaby/"
exports_location = root_location + "exported_data/"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

In [4]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 29.3 s, sys: 1.54 s, total: 30.8 s
Wall time: 30.8 s


In [5]:
len(training_docs_list)

1286325

#### Extraction Utils 

In [6]:
#ES_URL = 'http://localhost:9200/patents/patent/{}'
ES_URL = 'http://yell.dbs.ifi.lmu.de:9200/patents/patent/{}'
HEADING_TAG = 'heading'
PARAGRAPH_TAG = 'p'
UL_TAG = 'ul'
LI_TAG = 'li'
OL_TAG = 'ol'
DESC_OF_DRAWINGS_TAG = 'description-of-drawings'
MIN_PARAGRAPH_LENGTH = 50

In [7]:
def merge_with_previous(curr_node_tag, previous_node_tag, previous_node_text):
    if curr_node_tag == PARAGRAPH_TAG and previous_node_tag == HEADING_TAG:
        return True
    if previous_node_text and len(previous_node_text) < MIN_PARAGRAPH_LENGTH:
        return True
    return False
    
def get_paragraphs(root):
    paragraphs = []
    previous_node_text = None
    previous_tag = None
    for child in root:
        node_text = None
        if child.tag != DESC_OF_DRAWINGS_TAG:
            node_text = get_node_text(child)
            if node_text.strip():
                if merge_with_previous(child.tag, previous_tag, previous_node_text) and len(paragraphs) > 0:
                    paragraphs[-1] += ' ' + node_text
                else:
                    paragraphs.append(node_text)
        else:
            node_text = extract_desc_of_drawings_paragraph(child)
            paragraphs.append(node_text)
            
        previous_tag = child.tag
        previous_node_text = node_text
    return paragraphs
    
def extract_desc_of_drawings_paragraph(node):
    previous_tag = None
    sentences = []
    for child in node:
        node_text = get_node_text(child)
        if child.tag == PARAGRAPH_TAG and previous_tag == HEADING_TAG:
            sentences[-1] += ' ' + node_text
        else:
            # a paragraph in drawings descriptions is treated as a sentence
            if child.tag == PARAGRAPH_TAG:
                node_text = apply_sentence_end(node_text)
            sentences.append(node_text)
        previous_tag = child.tag
    
    return ' '.join(sentences)

def apply_sentence_end(text):
    if text and text.strip():
        text = text.strip().strip(';.')
        text += '. '
    return text

def itertext_custom(self):
    tag = self.tag
    if not isinstance(tag, basestring) and tag is not None:
        return
    if self.text:
        if tag == LI_TAG:
            yield apply_sentence_end(self.text)
        else:
            yield self.text.replace('\n',' ')
    for e in self:
        for s in e.itertext_custom():
            yield s
        if e.tail:
            yield e.tail

ET.Element.itertext_custom = itertext_custom
# def get_node_text(node):
#     node_text = ''
#     for child in node:
#         # for ul tags, get li tags as sentences
#         if child.tag == UL_TAG:
#             li_sentences = [apply_sentence_end(get_node_text_iterative(c)) for c in child]
#             child_text = ' '.join(li_sentences)
#         else:
#             child_text = get_node_text_iterative(child)
#         node_text += child_text
#     return node_text
        
get_node_text = lambda node: ''.join(node.itertext_custom()).strip()

In [8]:
def conc_paragraphs(parag1, parag2):
    return parag1.strip('.') + '.' + ' ' + parag2

def concatenate_sentences_to_paragraphs(paragraphs):
    """
    for 1 sentence paragraphs, concatenate them to the next or previous paragraph depending on context
    """
    for i in range(len(paragraphs)):
        if i >= len((paragraphs)): break
        parag = paragraphs[i]
        sentences = get_sentences(parag)
        
        if len(sentences) == 1:
            prev_paragraph = paragraphs[i-1] if i-1 >= 0 else None
            next_paragraph = paragraphs[i+1] if i+1 < len(paragraphs) else None

            if (next_paragraph and len(get_sentences(next_paragraph)) == 1):
                # If a series of 1 sentence length paragraphs exist, conc all of them in one paragraph
                while True:
                    if next_paragraph and len(get_sentences(next_paragraph)) == 1:
                        parag = conc_paragraphs(parag, next_paragraph)
                        paragraphs[i] = parag
                        del paragraphs[i+1]

                        # reinitialize for loop
                        next_paragraph = paragraphs[i+1] if i+1 < len(paragraphs) else None
                    else:
                        break

            # otherwise, just concatenate the 1 sentence paragraph to the previous paragraph
            elif prev_paragraph:
#                 print '============== Found prev eligible paragraph'
                prev_paragraph = conc_paragraphs(prev_paragraph, parag)
                paragraphs[i-1] = prev_paragraph
                del paragraphs[i]

            # if this is the first paragraph, then just concatenate it with the next one
            elif next_paragraph:
                parag = conc_paragraphs(parag, next_paragraph)
                paragraphs[i] = parag
                del paragraphs[i+1]

def get_adjusted_paragraphs(root, conc_sentences=True):
    paragraphs = get_paragraphs(root)
    if conc_sentences:
        concatenate_sentences_to_paragraphs(paragraphs)
    return paragraphs

In [9]:
def get_patent(doc_id):
    url_to_fetch = ES_URL.format(doc_id)

    response = urllib2.urlopen(url_to_fetch, timeout=60)
    patent_content = response.read()

    patent_object = json.loads(patent_content)['_source']
    return patent_object

# Actual Extraction

In [10]:
ABSTRACT_ID = "{}_abstract"
DESC_ID = "{}_description"
CLAIMS_ID = "{}_claims"

ABSTRACT_PART_ID = "{}_abstract_part-{}"
DESC_PART_ID = "{}_description_part-{}"
CLAIMS_PART_ID = "{}_claims_part-{}"

In [11]:
BATCH_SIZE = 10000
REPORT_EVERY = 1000

# preprocessed_location = "/mnt/data/shalaby/" + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"
preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"
TRAINING_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
TEST_PREPROCESSED_FILES_PREFIX = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

if not os.path.exists(preprocessed_location):
    os.makedirs(preprocessed_location)

In [12]:
NUM_ABSTRACT_PARTS = 3
NUM_DESC_PARTS = 23
NUM_CLAIMS_PARTS = 4

In [13]:
def multithreaded_extended_batch_creation(start_index):

    if os.path.exists(FILE_PREFIX + str(start_index)):
        info("Batch {} already exists, skipping..".format(start_index))
        return
    
    info("Batch creation working on {}\n".format(start_index))
    token_lines = []
    start_time = time.time()
    
    for doc_index, doc_id in enumerate(DOCS_LIST[start_index:]):
        if doc_index > 9900:
            print doc_index, doc_id
            
        patent_doc = get_patent(doc_id)
        
        # Abstract
        abstract = patent_doc['abstract'][0]
        root = ET.fromstring(abstract.encode('utf-8'))
        abs_paragraphs = get_adjusted_paragraphs(root)
        
        # Description
        desc = patent_doc['description'][0]
        root = ET.fromstring(desc.encode('utf-8'))
        desc_paragraphs = get_adjusted_paragraphs(root)
        
        # Claims
        claims = patent_doc['claims'][0]
        root = ET.fromstring(claims.encode('utf-8'))
        claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
#         claims_paragraphs = []
#         for claim in patent_doc['claims']:
#             claims_paragraphs.append(claim.strip())

#         abstract_sentences = sum([get_sentences(abs_parag) for abs_parag in abs_paragraphs], [])
#         desc_sentences = sum([get_sentences(desc_parag) for desc_parag in desc_paragraphs], [])
#         claims_sentences = sum([get_sentences(claim_parag) for claim_parag in claims_paragraphs], [])
        

        abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
        desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
        claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])
        
        
        
        # lists of list of tokens
        doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
        abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
        description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
        claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens
        
        # now add the tokens lists that will be written to the file
        token_lines.append(doc_tokens_list)
        token_lines.append(abstract_tokens_list)
        token_lines.append(description_tokens_list)
        token_lines.append(claims_tokens_list)
        
        for i in range(NUM_ABSTRACT_PARTS):
            start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
            token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])
        
        for i in range(NUM_DESC_PARTS):
            start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
            token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    
        
        for i in range(NUM_CLAIMS_PARTS):
            start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
            token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])
            
        if doc_index % 1000 == 0: info("Doc: {:6} -> Total Lines to write: {:8}".format(start_index + doc_index, len(token_lines)))
        if doc_index >= BATCH_SIZE - 1:
            break
        if doc_index >=9951:
            break
    duration = time.time() - start_time
    info("Finished batch {} of size {:d} in {:.0f}m {:.0f}s".format(start_index, BATCH_SIZE, * divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    
    write_batch(FILE_PREFIX, token_lines, start_index)
    del token_lines
    
def get_doc_range(i, number_of_tokens, number_of_parts):
    start, end = 0,0
    if number_of_tokens < number_of_parts:
        if i==0:
            return 0, None
        else:
            return number_of_tokens,None
    if i == 0:
        start = 0
    else:
        start = (number_of_tokens / number_of_parts) * i
    if i+1 == number_of_parts:
        end = None
    else:
        end = (number_of_tokens / number_of_parts) * (i+1)
    return start, end

In [14]:
def write_batch(file_prefix, batch_lines, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        with open(file_prefix + str(batch_start), 'w') as batch_file:
            for line in batch_lines:
                batch_file.write((u" ".join(line) + "\n").encode('utf-8'))
        print "finished writing batch %d" % batch_start

## Training

In [15]:
DOCS_LIST = training_docs_list
FILE_PREFIX = TRAINING_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(training_docs_list)

In [21]:
batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE)

In [22]:
batches

[0,
 10000,
 20000,
 30000,
 40000,
 50000,
 60000,
 70000,
 80000,
 90000,
 100000,
 110000,
 120000,
 130000,
 140000,
 150000,
 160000,
 170000,
 180000,
 190000,
 200000,
 210000,
 220000,
 230000,
 240000,
 250000]

In [23]:
try:
    pool = ThreadPool(9)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-22 00:57:06,749 : INFO : Batch creation working on 20000

2017-03-22 00:57:06,752 : INFO : Batch creation working on 40000

2017-03-22 00:57:06,750 : INFO : Batch creation working on 30000

2017-03-22 00:57:06,753 : INFO : Batch creation working on 50000

2017-03-22 00:57:06,749 : INFO : Batch creation working on 10000

2017-03-22 00:57:06,754 : INFO : Batch creation working on 80000

2017-03-22 00:57:06,753 : INFO : Batch creation working on 60000

2017-03-22 00:57:06,749 : INFO : Batch creation working on 0

2017-03-22 00:57:06,754 : INFO : Batch creation working on 70000

2017-03-22 00:57:13,067 : INFO : Doc:      0 -> Total Lines to write:       34
2017-03-22 00:57:13,087 : INFO : Doc:  30000 -> Total Lines to write:       34
2017-03-22 00:57:13,101 : INFO : Doc:  40000 -> Total Lines to write:       34
2017-03-22 00:57:13,114 : INFO : Doc:  60000 -> Total Lines to write:       34
2017-03-22 00:57:13,217 : INFO : Doc:  10000 -> Total Lines to write:       34
2017-03-22 00:5

writing batch 0


2017-03-22 01:34:39,276 : INFO : Doc:  69000 -> Total Lines to write:   306034
2017-03-22 01:34:55,892 : INFO : Doc:  89000 -> Total Lines to write:   306034
2017-03-22 01:35:05,018 : INFO : Batch creation working on 90000

2017-03-22 01:35:05,238 : INFO : Doc:  90000 -> Total Lines to write:       34
2017-03-22 01:36:18,382 : INFO : Doc:  79000 -> Total Lines to write:   306034
2017-03-22 01:36:58,889 : INFO : Finished batch 50000 of size 10000 in 39m 52s
2017-03-22 01:36:58,892 : INFO : For index 50000, the actual number of lines written is: 340000


writing batch 50000


2017-03-22 01:37:23,118 : INFO : Finished batch 30000 of size 10000 in 40m 16s
2017-03-22 01:37:23,121 : INFO : For index 30000, the actual number of lines written is: 340000


...

writing batch 220000


2017-03-22 03:03:03,237 : INFO : Finished batch 230000 of size 10000 in 40m 45s
2017-03-22 03:03:03,240 : INFO : For index 230000, the actual number of lines written is: 340000


writing batch 230000


2017-03-22 03:04:34,720 : INFO : Finished batch 240000 of size 10000 in 41m 24s
2017-03-22 03:04:34,723 : INFO : For index 240000, the actual number of lines written is: 340000


writing batch 240000


In [25]:
multithreaded_extended_batch_creation(0)

2017-03-16 10:44:02,083 : INFO : Batch creation working on 0

2017-03-16 10:44:02,175 : INFO : Doc:      0 -> Total Lines to write:       78
2017-03-16 10:44:18,155 : INFO : Finished batch 0 of size 100 in 0m 16s
2017-03-16 10:44:18,160 : INFO : For index 0, the actual number of lines written is: 7800


writing batch 0


## Validation

In [15]:
DOCS_LIST = validation_docs_list
FILE_PREFIX = VALIDATION_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(validation_docs_list)

In [ ]:
try:
    pool = ThreadPool(6)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-04-09 01:56:59,587 : INFO : Batch creation working on 60000

2017-04-09 01:56:59,588 : INFO : Batch creation working on 70000

2017-04-09 01:56:59,745 : INFO : Doc:  70000 -> Total Lines to write:       34
2017-04-09 01:56:59,761 : INFO : Doc:  60000 -> Total Lines to write:       34
2017-04-09 01:58:45,475 : INFO : Doc:  61000 -> Total Lines to write:    34034
2017-04-09 01:58:51,049 : INFO : Doc:  71000 -> Total Lines to write:    34034
2017-04-09 02:00:32,805 : INFO : Doc:  62000 -> Total Lines to write:    68034
2017-04-09 02:00:52,080 : INFO : Doc:  72000 -> Total Lines to write:    68034
2017-04-09 02:02:37,266 : INFO : Doc:  63000 -> Total Lines to write:   102034
2017-04-09 02:03:01,649 : INFO : Doc:  73000 -> Total Lines to write:   102034
2017-04-09 02:04:49,226 : INFO : Doc:  64000 -> Total Lines to write:   136034
2017-04-09 02:05:01,760 : INFO : Doc:  74000 -> Total Lines to write:   136034
2017-04-09 02:06:59,091 : INFO : Doc:  65000 -> Total Lines to write:   170034

writing batch 70000


Process PoolWorker-3:
Process PoolWorker-4:
Process PoolWorker-5:
Process PoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self

In [20]:
%%time
ff = get_patent(validation_docs_list[69952])

CPU times: user 1.03 s, sys: 524 ms, total: 1.55 s
Wall time: 12.8 s


In [22]:
patent_doc = ff
doc_id = validation_docs_list[69952]

In [17]:
validation_docs_list[69952]

u'08207316'

In [39]:
validation_docs_list[69953]

u'08002075'

In [62]:
token_lines = []

In [63]:
doc_id = validation_docs_list[69951]
patent_doc = get_patent(doc_id)
print 'Downloaded Document'

# Abstract
abstract = patent_doc['abstract'][0]
root = ET.fromstring(abstract.encode('utf-8'))
abs_paragraphs = get_adjusted_paragraphs(root)
info('Finished abstract')

# Description
desc = patent_doc['description'][0]
root = ET.fromstring(desc.encode('utf-8'))
desc_paragraphs = get_adjusted_paragraphs(root)
info('Finished description')

# Claims
claims = patent_doc['claims'][0]
root = ET.fromstring(claims.encode('utf-8'))
claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
info('Finished claims')


abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])
info('Finished sentences')



# lists of list of tokens
doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens

# now add the tokens lists that will be written to the file
token_lines.append(doc_tokens_list)
token_lines.append(abstract_tokens_list)
token_lines.append(description_tokens_list)
token_lines.append(claims_tokens_list)

for i in range(NUM_ABSTRACT_PARTS):
    start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
    token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])

for i in range(NUM_DESC_PARTS):
    start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
    token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    

for i in range(NUM_CLAIMS_PARTS):
    start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
    token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])


2017-04-10 00:59:34,495 : INFO : Finished abstract
2017-04-10 00:59:34,571 : INFO : Finished description
2017-04-10 00:59:34,577 : INFO : Finished claims


Downloaded Document


2017-04-10 00:59:34,802 : INFO : Finished sentences


In [64]:
doc_id = validation_docs_list[69952]
patent_doc = get_patent(doc_id)
print 'Downloaded Document'

# Abstract
abstract = patent_doc['abstract'][0]
root = ET.fromstring(abstract.encode('utf-8'))
abs_paragraphs = get_adjusted_paragraphs(root)
info('Finished abstract')

# Description
desc = patent_doc['description'][0]
root = ET.fromstring(desc.encode('utf-8'))
desc_paragraphs = get_adjusted_paragraphs(root)
info('Finished description')

# Claims
claims = patent_doc['claims'][0]
root = ET.fromstring(claims.encode('utf-8'))
claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
info('Finished claims')


abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
# desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])
info('Finished sentences')



# # lists of list of tokens
# doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
# abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
# description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
# claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens

# # now add the tokens lists that will be written to the file
# token_lines.append(doc_tokens_list)
# token_lines.append(abstract_tokens_list)
# token_lines.append(description_tokens_list)
# token_lines.append(claims_tokens_list)

# for i in range(NUM_ABSTRACT_PARTS):
#     start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
#     token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])

# for i in range(NUM_DESC_PARTS):
#     start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
#     token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    

# for i in range(NUM_CLAIMS_PARTS):
#     start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
#     token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])


2017-04-10 00:59:56,967 : INFO : Finished abstract


Downloaded Document


2017-04-10 01:00:43,768 : INFO : Finished description
2017-04-10 01:00:43,770 : INFO : Finished claims
2017-04-10 01:00:43,775 : INFO : Finished sentences


In [65]:
desc_tokens = pickle.load(open(root_location + 'desc_tokens.pkl', 'r'))

In [66]:


# lists of list of tokens
doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens

# now add the tokens lists that will be written to the file
token_lines.append(doc_tokens_list)
token_lines.append(abstract_tokens_list)
token_lines.append(description_tokens_list)
token_lines.append(claims_tokens_list)

for i in range(NUM_ABSTRACT_PARTS):
    start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
    token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])

for i in range(NUM_DESC_PARTS):
    start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
    token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    

for i in range(NUM_CLAIMS_PARTS):
    start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
    token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])


In [67]:
len(token_lines)

68

In [58]:
len(claims_tokens)

863

In [28]:
pickle.dump(desc_tokens, open(root_location + 'desc_tokens.pkl', 'w'))

In [ ]:


# lists of list of tokens
doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens

# now add the tokens lists that will be written to the file
token_lines.append(doc_tokens_list)
token_lines.append(abstract_tokens_list)
token_lines.append(description_tokens_list)
token_lines.append(claims_tokens_list)

for i in range(NUM_ABSTRACT_PARTS):
    start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
    token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])

for i in range(NUM_DESC_PARTS):
    start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
    token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    

for i in range(NUM_CLAIMS_PARTS):
    start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
    token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])


In [35]:
len(token_lines)

1598

In [37]:
token_lines[0][:10]

[u'08002075',
 u'a',
 u'steering',
 u'device',
 u'for',
 u'a',
 u'superposition',
 u'steering',
 u'system',
 u'is']

In [20]:
multithreaded_extended_batch_creation(60000)

2017-04-09 15:46:24,554 : INFO : Batch creation working on 60000

2017-04-09 15:46:24,781 : INFO : Doc:  60000 -> Total Lines to write:       34
2017-04-09 15:50:50,363 : INFO : Doc:  61000 -> Total Lines to write:    34034
2017-04-09 15:53:49,752 : INFO : Doc:  62000 -> Total Lines to write:    68034
2017-04-09 15:57:19,548 : INFO : Doc:  63000 -> Total Lines to write:   102034
2017-04-09 16:01:08,497 : INFO : Doc:  64000 -> Total Lines to write:   136034
2017-04-09 16:05:12,110 : INFO : Doc:  65000 -> Total Lines to write:   170034
2017-04-09 16:08:59,667 : INFO : Doc:  66000 -> Total Lines to write:   204034
2017-04-09 16:13:06,318 : INFO : Doc:  67000 -> Total Lines to write:   238034
2017-04-09 16:17:09,561 : INFO : Doc:  68000 -> Total Lines to write:   272034
2017-04-09 16:20:55,102 : INFO : Doc:  69000 -> Total Lines to write:   306034


9901 08124989
9902 07256083
9903 07156768
9904 07256081
9905 08077101
9906 08947185
9907 08344608
9908 08135716
9909 08832241
9910 07868387
9911 07765739
9912 08201026
9913 07157419
9914 08375068
9915 08077671
9916 08220098
9917 08220097
9918 08220096
9919 07410901
9920 07689870
9921 07765734
9922 07157414
9923 07004038
9924 07299058
9925 08862622
9926 07526627
9927 08605647
9928 08690261
9929 08937442
9930 08908705
9931 07871290
9932 08220331
9933 07668057
9934 08800629
9935 08102248
9936 07177754
9937 07177755
9938 08791969
9939 07141621
9940 07374676
9941 08791966
9942 07374675
9943 07374672
9944 08791962
9945 07548519
9946 08508349
9947 08488117
9948 08212152
9949 08723448
9950 08207318
9951 07612796


2017-04-09 16:24:28,111 : INFO : Finished batch 60000 of size 10000 in 38m 4s
2017-04-09 16:24:28,113 : INFO : For index 60000, the actual number of lines written is: 338368


writing batch 60000
finished writing batch 60000


In [54]:
validation_docs_list[69950: 69953]

[u'08207318', u'07612796', u'08207316']

In [51]:
validation_docs_list[69952]

u'08207316'

In [68]:
start_index = 69953
info("Batch creation working on {}\n".format(start_index))
start_time = time.time()

for doc_index, doc_id in enumerate(DOCS_LIST[start_index:70000]):
    print doc_index, doc_id

    patent_doc = get_patent(doc_id)

    # Abstract
    abstract = patent_doc['abstract'][0]
    root = ET.fromstring(abstract.encode('utf-8'))
    abs_paragraphs = get_adjusted_paragraphs(root)

    # Description
    desc = patent_doc['description'][0]
    root = ET.fromstring(desc.encode('utf-8'))
    desc_paragraphs = get_adjusted_paragraphs(root)

    # Claims
    claims = patent_doc['claims'][0]
    root = ET.fromstring(claims.encode('utf-8'))
    claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
#         claims_paragraphs = []
#         for claim in patent_doc['claims']:
#             claims_paragraphs.append(claim.strip())

#         abstract_sentences = sum([get_sentences(abs_parag) for abs_parag in abs_paragraphs], [])
#         desc_sentences = sum([get_sentences(desc_parag) for desc_parag in desc_paragraphs], [])
#         claims_sentences = sum([get_sentences(claim_parag) for claim_parag in claims_paragraphs], [])


    abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
    desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
    claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])


    # lists of list of tokens
    doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
    abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
    description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
    claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens

    # now add the tokens lists that will be written to the file
    token_lines.append(doc_tokens_list)
    token_lines.append(abstract_tokens_list)
    token_lines.append(description_tokens_list)
    token_lines.append(claims_tokens_list)

    for i in range(NUM_ABSTRACT_PARTS):
        start, end = get_doc_range(i, len(abstract_tokens), NUM_ABSTRACT_PARTS)
        token_lines.append([ABSTRACT_PART_ID.format(doc_id, i+1)] + abstract_tokens[start: end])

    for i in range(NUM_DESC_PARTS):
        start, end = get_doc_range(i, len(desc_tokens), NUM_DESC_PARTS)
        token_lines.append([DESC_PART_ID.format(doc_id, i+1)] + desc_tokens[start: end])    

    for i in range(NUM_CLAIMS_PARTS):
        start, end = get_doc_range(i, len(claims_tokens), NUM_CLAIMS_PARTS)
        token_lines.append([CLAIMS_PART_ID.format(doc_id, i+1)] + claims_tokens[start: end])

duration = time.time() - start_time
info("Finished batch {} of size {:d} in {:.0f}m {:.0f}s".format(start_index, BATCH_SIZE, * divmod(duration, 60)))
info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))


2017-04-10 01:11:37,344 : INFO : Batch creation working on 69953



0 08002075
1 08457003
2 08811420
3 08235497
4 08872443
5 08036304
6 08274294
7 07612300
8 07672841
9 07226675
10 08920199
11 07265847
12 07051244
13 07993771
14 07600946
15 07051248
16 08121513
17 07890307
18 07644315
19 07444309
20 07644317
21 07826008
22 07877085
23 07472013
24 08771000
25 07444300
26 07444301
27 07956763
28 08276676
29 08465626
30 07354709
31 08903939
32 08171054
33 08294036
34 07400332
35 08033773
36 07348728
37 07460605
38 07786687
39 07052574
40 07194614
41 07167804
42 07052577
43 08391069
44 08085626
45 07052572


2017-04-10 01:11:48,342 : INFO : Finished batch 69953 of size 10000 in 0m 11s
2017-04-10 01:11:48,344 : INFO : For index 69953, the actual number of lines written is: 1666


46 07391325


In [69]:
len(token_lines)

1666

In [70]:
with open(VALIDATION_PREPROCESSED_FILES_PREFIX + str(60000), 'a+') as batch_file:
    for line in token_lines:
        batch_file.write((u" ".join(line) + "\n").encode('utf-8'))

## Test

In [15]:
DOCS_LIST = test_docs_list
FILE_PREFIX = TEST_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(test_docs_list)

In [16]:
try:
    pool = ThreadPool(8)
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-04-07 00:57:25,762 : INFO : Batch creation working on 40000

2017-04-07 00:57:25,761 : INFO : Batch creation working on 20000

2017-04-07 00:57:25,761 : INFO : Batch creation working on 0

2017-04-07 00:57:25,762 : INFO : Batch creation working on 100000

2017-04-07 00:57:25,763 : INFO : Batch creation working on 60000

2017-04-07 00:57:25,762 : INFO : Batch creation working on 80000

2017-04-07 00:57:25,763 : INFO : Batch creation working on 120000

2017-04-07 00:57:25,763 : INFO : Batch creation working on 140000

2017-04-07 00:57:26,605 : INFO : Doc: 100000 -> Total Lines to write:       34
2017-04-07 00:57:26,662 : INFO : Doc:  40000 -> Total Lines to write:       34
2017-04-07 00:57:26,678 : INFO : Doc:  80000 -> Total Lines to write:       34
2017-04-07 00:57:26,718 : INFO : Doc:  20000 -> Total Lines to write:       34
2017-04-07 00:57:26,841 : INFO : Doc: 120000 -> Total Lines to write:       34
2017-04-07 00:57:26,890 : INFO : Doc:      0 -> Total Lines to write:       34

writing batch 20000


2017-04-07 01:39:58,210 : INFO : Batch creation working on 30000

2017-04-07 01:39:58,586 : INFO : Doc:  30000 -> Total Lines to write:       34
2017-04-07 01:40:34,885 : INFO : Finished batch 80000 of size 10000 in 43m 9s
2017-04-07 01:40:34,887 : INFO : For index 80000, the actual number of lines written is: 340000


writing batch 80000


2017-04-07 01:40:59,764 : INFO : Finished batch 100000 of size 10000 in 43m 34s
2017-04-07 01:40:59,767 : INFO : For index 100000, the actual number of lines written is: 340000


...

writing batch 350000


2017-04-07 05:06:47,331 : INFO : Finished batch 390000 of size 10000 in 34m 41s
2017-04-07 05:06:47,334 : INFO : For index 390000, the actual number of lines written is: 340000


writing batch 390000


2017-04-07 05:07:54,237 : INFO : Finished batch 370000 of size 10000 in 35m 36s
2017-04-07 05:07:54,241 : INFO : For index 370000, the actual number of lines written is: 340000


writing batch 370000


## Stats

In [37]:
BATCH_SIZE = 1000

In [38]:
def multithreaded_extended_batch_creation(start_index):

#     if os.path.exists(FILE_PREFIX + str(start_index)):
#         info("Batch {} already exists, skipping..".format(start_index))
#         return
    
    info("Batch creation working on {}\n".format(start_index))
    token_lines = []
    start_time = time.time()
    
    len_abs_sentences = 0
    len_desc_sentences = 0
    len_desc_paragraphs = 0
    len_claims_sentences = 0
    
    len_abs_tokens = []
    len_desc_tokens = []
    len_claims_tokens = []
    
    
    len_desc_parag_tokens = []

    len_claims_paragraphs = []
    
    for doc_index, doc_id in enumerate(DOCS_LIST[start_index:]):
        patent_doc = get_patent(doc_id)
        
        # Abstract
        abstract = patent_doc['abstract'][0]
        root = ET.fromstring(abstract.encode('utf-8'))
        abs_paragraphs = get_adjusted_paragraphs(root)
        
        # Description
        desc = patent_doc['description'][0]
        root = ET.fromstring(desc.encode('utf-8'))
        desc_paragraphs = get_adjusted_paragraphs(root)
        
        # Claims
        claims = patent_doc['claims'][0]
        root = ET.fromstring(claims.encode('utf-8'))
        claims_paragraphs = get_adjusted_paragraphs(root, conc_sentences=False)
#         claims_paragraphs = []
#         for claim in patent_doc['claims']:
#             claims_paragraphs.append(claim.strip())

        len_claims_paragraphs.append(len(claims_paragraphs))
    
        abstract_sentences = sum([get_sentences(abs_parag) for abs_parag in abs_paragraphs], [])
        desc_sentences = sum([get_sentences(desc_parag) for desc_parag in desc_paragraphs], [])
        claims_sentences = sum([get_sentences(claim_parag) for claim_parag in claims_paragraphs], [])
        
        len_abs_sentences += len(abstract_sentences)
        len_desc_sentences += len(desc_sentences)
        len_claims_sentences += len(claims_sentences)
        
        len_desc_paragraphs += len(desc_paragraphs)
        

        abstract_tokens = sum([sentence_wordtokenizer(parag) for parag in abs_paragraphs], [])
        
        len_desc_parag_tokens.extend([len(sentence_wordtokenizer(parag)) for parag in desc_paragraphs])
        desc_tokens = sum([sentence_wordtokenizer(parag) for parag in desc_paragraphs], [])
        
        claims_tokens = sum([sentence_wordtokenizer(parag) for parag in claims_paragraphs], [])
        
        
        len_abs_tokens.append(len(abstract_tokens))
        len_desc_tokens.append(len(desc_tokens))
        len_claims_tokens.append(len(claims_tokens))
        
        # lists of list of tokens
        doc_tokens_list = [doc_id]  + abstract_tokens + desc_tokens + claims_tokens
        abstract_tokens_list = [ABSTRACT_ID.format(doc_id)] + abstract_tokens
        description_tokens_list = [DESC_ID.format(doc_id)] + desc_tokens
        claims_tokens_list = [CLAIMS_ID.format(doc_id)] + claims_tokens
        
        # now add the tokens lists that will be written to the file
        token_lines.append(doc_tokens_list)
        token_lines.append(abstract_tokens_list)
        token_lines.append(description_tokens_list)
        token_lines.append(claims_tokens_list)
        
        if doc_index % 1000 == 0: info("Doc: {:6} -> Total Lines to write: {:8}".format(start_index + doc_index, len(token_lines)))
        if doc_index >= BATCH_SIZE - 1:
            break
    duration = time.time() - start_time
    info("Finished batch {} of size {:d} in {:.0f}m {:.0f}s".format(start_index, BATCH_SIZE, * divmod(duration, 60)))
    info("For index {}, the actual number of lines written is: {}".format(start_index, len(token_lines)))
    
    print "Average Abstract Sentences: {}".format(len_abs_sentences/doc_index)
    print "Average Desc Sentences: {}".format(len_desc_sentences/doc_index)
    print "Average Desc Paragraphs: {}".format(len_desc_paragraphs/doc_index)
    print "Average Claims Sentences: {}".format(len_claims_sentences/doc_index)
    
    
    print "Abstract Tokens: Mean: {} - Median: {}".format(np.mean(len_abs_tokens), np.median(len_abs_tokens))
    print "Description Tokens: Mean: {} - Median: {}".format(np.mean(len_desc_tokens), np.median(len_desc_tokens))
    print "Claims Tokens: Mean: {} - Median: {}".format(np.mean(len_claims_tokens), np.median(len_claims_tokens))
    print "Description Paragraphs Tokens: Mean: {} - Median: {}".format(np.mean(len_desc_parag_tokens), 
                                                                        np.median(len_desc_parag_tokens))
    
    print "Claims Paragraphs: Mean: {} - Median: {}".format(np.mean(len_claims_paragraphs), 
                                                                        np.median(len_claims_paragraphs))
#     write_batch(FILE_PREFIX, token_lines, start_index)
    del token_lines

In [39]:
DOCS_LIST = training_docs_list
FILE_PREFIX = TRAINING_PREPROCESSED_FILES_PREFIX
SAMPLE_SIZE = len(training_docs_list)

In [40]:
try:
    pool = ThreadPool(8) # use just 6 because every batch requires a lot of memory
    # +1 since range is end-exclusive
    batches = range(0, (divmod(SAMPLE_SIZE, BATCH_SIZE)[0]+1) * BATCH_SIZE, BATCH_SIZE*10 )
    indices = pool.map(multithreaded_extended_batch_creation, batches)
    pool.close()
    pool.terminate()
finally:
    pool.close()
    pool.terminate()

2017-03-16 01:00:43,555 : INFO : Batch creation working on 0

2017-03-16 01:00:43,555 : INFO : Batch creation working on 10000

2017-03-16 01:00:43,558 : INFO : Batch creation working on 40000

2017-03-16 01:00:43,555 : INFO : Batch creation working on 20000

2017-03-16 01:00:43,558 : INFO : Batch creation working on 50000

2017-03-16 01:00:43,559 : INFO : Batch creation working on 30000

2017-03-16 01:00:43,591 : INFO : Batch creation working on 60000

2017-03-16 01:00:43,601 : INFO : Batch creation working on 70000

2017-03-16 01:00:43,690 : INFO : Doc:  50000 -> Total Lines to write:        4
2017-03-16 01:00:43,742 : INFO : Doc:  60000 -> Total Lines to write:        4
2017-03-16 01:00:43,767 : INFO : Doc:      0 -> Total Lines to write:        4
2017-03-16 01:00:43,806 : INFO : Doc:  40000 -> Total Lines to write:        4
2017-03-16 01:00:43,846 : INFO : Doc:  20000 -> Total Lines to write:        4
2017-03-16 01:00:43,904 : INFO : Doc:  10000 -> Total Lines to write:        4
20

Average Abstract Sentences: 3
Average Desc Sentences: 236
Average Desc Paragraphs: 56
Average Claims Sentences: 19
Abstract Tokens: Mean: 118.576 - Median: 114.0
Description Tokens: Mean: 7694.354 - Median: 5256.5
Claims Tokens: Mean: 1101.352 - Median: 828.0
Description Paragraphs Tokens: Mean: 137.372194747 - Median: 113.0
Claims Paragraphs: Mean: 19.411 - Median: 16.0


2017-03-16 01:05:31,749 : INFO : Batch creation working on 80000

2017-03-16 01:05:33,147 : INFO : Doc:  80000 -> Total Lines to write:        4
2017-03-16 01:05:49,041 : INFO : Finished batch 40000 of size 1000 in 5m 5s
2017-03-16 01:05:49,060 : INFO : For index 40000, the actual number of lines written is: 4000


Average Abstract Sentences: 3
Average Desc Sentences: 261
Average Desc Paragraphs: 61
Average Claims Sentences: 17
Abstract Tokens: Mean: 119.666 - Median: 119.0
Description Tokens: Mean: 8411.443 - Median: 6107.0
Claims Tokens: Mean: 1111.596 - Median: 944.0
Description Paragraphs Tokens: Mean: 137.076788944 - Median: 113.0
Claims Paragraphs: Mean: 17.226 - Median: 16.0


2017-03-16 01:05:49,886 : INFO : Batch creation working on 90000

2017-03-16 01:05:50,033 : INFO : Doc:  90000 -> Total Lines to write:        4
2017-03-16 01:05:56,244 : INFO : Finished batch 50000 of size 1000 in 5m 13s
2017-03-16 01:05:56,248 : INFO : For index 50000, the actual number of lines written is: 4000


...

Average Abstract Sentences: 3
Average Desc Sentences: 276
Average Desc Paragraphs: 65
Average Claims Sentences: 16
Abstract Tokens: Mean: 116.039 - Median: 118.0
Description Tokens: Mean: 9215.904 - Median: 6593.5
Claims Tokens: Mean: 1066.299 - Median: 932.5
Description Paragraphs Tokens: Mean: 141.239908046 - Median: 116.0
Claims Paragraphs: Mean: 16.451 - Median: 16.0


2017-03-16 01:11:36,285 : INFO : Finished batch 100000 of size 1000 in 5m 39s
2017-03-16 01:11:36,288 : INFO : For index 100000, the actual number of lines written is: 4000


Average Abstract Sentences: 3
Average Desc Sentences: 287
Average Desc Paragraphs: 67
Average Claims Sentences: 16
Abstract Tokens: Mean: 113.675 - Median: 117.0
Description Tokens: Mean: 9544.564 - Median: 6973.5
Claims Tokens: Mean: 1108.746 - Median: 977.5
Description Paragraphs Tokens: Mean: 141.208486211 - Median: 116.0
Claims Paragraphs: Mean: 16.438 - Median: 17.0


2017-03-16 01:11:54,205 : INFO : Finished batch 110000 of size 1000 in 5m 55s
2017-03-16 01:11:54,208 : INFO : For index 110000, the actual number of lines written is: 4000


Average Abstract Sentences: 3
Average Desc Sentences: 311
Average Desc Paragraphs: 73
Average Claims Sentences: 16
Abstract Tokens: Mean: 114.923 - Median: 118.0
Description Tokens: Mean: 10124.423 - Median: 7225.5
Claims Tokens: Mean: 1053.8 - Median: 936.5
Description Paragraphs Tokens: Mean: 138.761056974 - Median: 115.0
Claims Paragraphs: Mean: 16.433 - Median: 16.0
